In [26]:
from dolfin import *
from mshr import *

w = 0.01
l = 1.
tol = 10E-14

def solve_problem(lambd, R, count):
    mesh = generate_mesh(Circle(Point(0.,0.), R, 30) - Rectangle(Point(-l, -w), Point(l, w)), count)

    class OuterBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and (near(abs(x[0]), R) or near(abs(x[1]), R))

    class InnerBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol

    Gamma_N = InnerBoundary()

    boundary_adaptor = EdgeFunction('bool', mesh)
    Gamma_N.mark(boundary_adaptor, True)
    for i in xrange(1):
        mesh = adapt(mesh, boundary_adaptor)

    boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)
    

    Gamma_R = OuterBoundary()
    Gamma_R.mark(boundary_parts, 0)

    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [#DirichletBC(V, Expression('-lambd * log(x[0]*x[0]+x[1]*x[1]) / 2.', lambd=lambd), OuterBoundary()),
           DirichletBC(V, Constant(1.), InnerBoundary())]

    a = inner(grad(u), grad(v)) * dx
    #L = Constant(0.0) * v * ds
    dS = ds(subdomain_data=boundary_parts)
    L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)
    
    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()
    
    return u, mesh

In [23]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri

def contour_plot(u, mesh, levels_count):
    triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                               triangles=mesh.cells())
    Z = u.compute_vertex_values(mesh)

    plt.figure()
    plt.tricontour(triang, Z, levels_count)
    plt.colorbar()
    plt.show()

In [27]:
def find_lambda(lambda_range, point, u_point, R, count, tol=10E-10):
    l, r = lambda_range
    lambd = (l + r) / 2
    u_point_numerical = solve_problem(lambd, R, count)[0](*point)
    while abs(u_point_numerical - u_point) > tol:
        if u_point_numerical - u_point > 0:
            r = lambd
        else:
            l = lambd
        lambd = (l + r) / 2
        print lambd
        u_point_numerical = solve_problem(lambd, R, count)[0](*point)
        print 'value:', u_point_numerical
    return lambd

In [29]:
lambda_range = (-0.95, 0.)
lambd = find_lambda(lambda_range, (10., 0.), 0., 30., 100, tol =10E-4)
print 'lambda:', lambd

-0.7125
value: -0.0937141598964
-0.59375
value: 0.808256844929
-0.653125
value: 0.996598993432
-0.6828125
value: 0.973925052272
-0.69765625
value: -0.0709284482319
-0.690234375
value: 0.845399094419
-0.6939453125
value: 0.963596578569
-0.69580078125
value: 0.985291180662
-0.696728515625
value: 0.0333614172568
-0.697192382812
value: 1.0
-0.697424316406
value: 0.847712085212
-0.697540283203
value: -0.0604046900678
-0.697482299805
value: -0.0706614281733
-0.697453308105
value: 0.625895830061
-0.697467803955
value: -0.116856555001
-0.69746055603
value: -0.116844948885
-0.697456932068
value: 1.0
-0.697458744049
value: 1.0
-0.69745965004
value: 0.736182025867
-0.697460103035
value: 1.0
-0.697460329533
value: -0.0650990535728
-0.697460216284
value: 0.896070881885
-0.697460272908
value: 0.918060711639
-0.69746030122
value: 0.434402798071
-0.697460315377
value: 0.100820520027
-0.697460322455
value: -0.0706276921186
-0.697460318916
value: -0.06123863631
-0.697460317146
value: 1.0
-0.697460318031

KeyboardInterrupt: 

In [ ]:
lambd = -0.2939453125
u, mesh = solve_problem(lambd, 30., 100)

In [ ]:
plot(u, interactive=True)
contour_plot(u, mesh, 30)

In [ ]:
import math

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(5. / l)
        B = 1.
        z = complex(x[0], x[1])
        ln = abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) )
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

In [ ]:
plot(mesh, interactive=True)
V = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), V)

In [30]:
from dolfin import *
from mshr import *

w = 0.005
l = 1.
tol = 10E-14
R = 30.
count = 100
lambd = -0.2939453125

mesh = generate_mesh(Circle(Point(0., 0.), R, 30) - Rectangle(Point(-l, -w), Point(l, w)), count)

class OuterBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (near(abs(x[0]), R) or near(abs(x[1]), R))

class InnerBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol

Gamma_N = InnerBoundary()

boundary_adaptor = EdgeFunction('bool', mesh)
Gamma_N.mark(boundary_adaptor, True)
for i in xrange(1):
    mesh = adapt(mesh, boundary_adaptor)

boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)


Gamma_R = OuterBoundary()
Gamma_R.mark(boundary_parts, 0)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(1.), InnerBoundary())]

a = inner(grad(u), grad(v)) * dx
#L = Constant(0.0) * v * ds
dS = ds(subdomain_data=boundary_parts)
L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(mesh)



import math
import cmath

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(10. / l)
        B = 1.
        z = complex(x[0], x[1])
        ln = math.log(abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) ))
        if x[0] < 0:
            ln *= -1.
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

Ve = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), Ve)

plot(u - exact)

print u(10., 0.), exact(10., 0.)

interactive()

0.717725787879 -4.1672560044e-09


In [31]:
contour_plot(u, mesh, 50)

In [21]:
from dolfin import *
from mshr import *

w = 0.005
l = 1.
tol = 10E-14
R = 30.
count = 100
lambd = -0.2939453125

mesh = generate_mesh(Circle(Point(0., 0.), R, 70) - Rectangle(Point(-R, -R), Point(0, R)) -
                     Rectangle(Point(0, -R), Point(R, 0)) - Rectangle(Point(0., -w), Point(l, w)), count)

class OuterBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and x[0] > 0 and x[1] > 0

class InnerBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol

Gamma_N = InnerBoundary()

boundary_adaptor = EdgeFunction('bool', mesh)
Gamma_N.mark(boundary_adaptor, True)
for i in xrange(1):
    mesh = adapt(mesh, boundary_adaptor)

boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)


Gamma_R = OuterBoundary()
Gamma_R.mark(boundary_parts, 0)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(1.), InnerBoundary())]

a = inner(grad(u), grad(v)) * dx
dS = ds(subdomain_data=boundary_parts)
L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(mesh)



import math
import cmath

class Exact_Solution(Expression):
    def eval(self, value, x):
        A = -1. / math.acosh(10. / l)
        B = 1.
        z = complex(x[0], x[1])
        ln = math.log(abs( z / l + cmath.sqrt( (z / l) ** 2 - 1 ) ))
        if x[0] < 0:
            ln *= -1.
        value[0] = A * ln + B
    def value_shape(self):
        return (1,)

Ve = FunctionSpace(mesh, 'Lagrange', 3)
exact = interpolate(Exact_Solution(element=FiniteElement("Quadrature", triangle, 2)), Ve)

plot(u)
plot(exact, mesh)
plot(u - exact)

print u(10., 0.), exact(10., 0.)

interactive()

-0.567241198284 1.05564139033e-10


In [12]:
from dolfin import *
from mshr import *

w = 0.005
l = 1.
tol = 10E-14
R = 30.
count = 100

mesh = generate_mesh(Circle(Point(0., 0.), R, 70) - Rectangle(Point(-R, -R), Point(0, R)) -
                         Rectangle(Point(0, -R), Point(R, 0)) - Rectangle(Point(0., -w), Point(l, w)), count)

def solve_p(lambd):
    global mesh
    
    class OuterBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and x[0] > 0 and x[1] > 0

    class InnerBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and abs(x[0]) < l - tol and abs(x[1]) < w + tol


    boundary_parts = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)


    Gamma_R = OuterBoundary()
    Gamma_R.mark(boundary_parts, 0)

    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [DirichletBC(V, Constant(1.), InnerBoundary())]

    a = inner(grad(u), grad(v)) * dx
    dS = ds(subdomain_data=boundary_parts)
    L = Expression('lambd / sqrt(x[0]*x[0] + x[1]*x[1])', lambd=lambd) * v * dS(0)

    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()

    return u

'''
    Gamma_N = InnerBoundary()

    boundary_adaptor = EdgeFunction('bool', mesh)
    Gamma_N.mark(boundary_adaptor, True)
    for i in xrange(1):
        mesh = adapt(mesh, boundary_adaptor)
'''

"\n    Gamma_N = InnerBoundary()\n\n    boundary_adaptor = EdgeFunction('bool', mesh)\n    Gamma_N.mark(boundary_adaptor, True)\n    for i in xrange(1):\n        mesh = adapt(mesh, boundary_adaptor)\n"

In [13]:
def find_lambda(lambda_range, point, u_point, tol=10E-10):
    l, r = lambda_range
    lambd = (l + r) / 2
    u_point_numerical = solve_p(lambd)(*point)
    while abs(u_point_numerical - u_point) > tol:
        if u_point_numerical - u_point > 0:
            r = lambd
        else:
            l = lambd
        lambd = (l + r) / 2
        print lambd
        u_point_numerical = solve_p(lambd)(*point)
        print 'value:', u_point_numerical
    return lambd

In [18]:
import numpy as np
lambda_range = (-0.19, -0.18)
#lambd = find_lambda(lambda_range, (10., 0.), 0.)
#print lambd
print [solve_p(a)(10., 0.) for a in np.linspace(-0.19, -0.18, 50)]

[0.33836817049601092, 0.43618175827581496, 0.43678801444971288, 0.43739427062361064, 0.43800052679750923, 0.43860678297140721, 0.27345652290395256, 0.27424197531162536, 0.44042555149310131, 0.44103180766700023, 0.44163806384089843, 0.44224432001479663, 0.44285057618869478, 0.44345683236259303, 0.44406308853649135, 0.28052559457300619, 0.44527560088428775, 0.28209649938835174, 0.44648811323208387, 0.44709436940598241, 0.44770062557988027, 0.28523830901904251, 0.44891313792767662, 0.44951939410157482, 0.28759466624205993, 0.45073190644937144, 0.28916557105740487, 0.45194441879716779, 0.29073647587275075, 0.4531569311449643, 0.45376318731886223, 0.45436944349276004, 0.45497569966665874, 0.45558195584055655, 0.29544919031878625, 0.45679446818835312, 0.29702009513413147, 0.45800698053614963, 0.29859099994947702, 0.45921949288394581, 0.45982574905784424, 0.4604320052317426, 0.46103826140564058, -0.16215478043754583, 0.30330371439551285, 0.46285702992733524, 0.30487461921085829, 0.46406954227